In [ ]:
!pip install onnx
!pip install onnxscript
!pip install onnxruntime

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.9/15.9 MB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 518.4/518.4 kB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 21.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 1.1 MB/s eta 0:00:00


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!git clone https://github.com/Mhaiyang/CVPR2020_GDNet.git

Cloning into 'CVPR2020_GDNet'...
remote: Enumerating objects: 67, done.
remote: Counting objects: 100% (37/37), done.
remote: Compressing objects: 100% (20/20), done.
remote: Total 67 (delta 17), reused 17 (delta 17), pack-reused 30
Receiving objects: 100% (67/67), 17.40 KiB | 1.02 MiB/s, done.
Resolving deltas: 100% (29/29), done.


In [ ]:
cd CVPR2020_GDNet/

/content/CVPR2020_GDNet


In [ ]:
import cv2
import numpy as np
import time
import onnx
import onnxruntime
import torchvision.transforms as transforms
from PIL import Image
import matplotlib.pyplot as plt
import torch
from torch.autograd import Variable
from gdnet import GDNet
# from MirrorNet.mirrornet import MirrorNet

model = GDNet()
model.load_state_dict(torch.load("/content/drive/MyDrive/200.pth"))
# model = MirrorNet()
# model.load_state_dict(torch.load("MirrorNet/ckpt/MirrorNet/160.pth"))
model.eval()

dummy_input = Variable(torch.randn(1, 3, 416, 416)).to("cuda")
# x = torch.randn(1, 3, 416, 416, requires_grad=True)
# x = torch.randn(1, 3, 384, 384, requires_grad=True)
model.to("cuda")

with torch.no_grad():
    #save to onnx
    torch.onnx.export(  model,               # model being run
                        dummy_input,                         # model input (or a tuple for multiple inputs)
                        "model_gdnet.onnx",   # where to save the model (can be a file or file-like object)
                        export_params=True,        # store the trained parameter weights inside the model file
                        opset_version=11,          # the ONNX version to export the model to 16-Mirrornet
                        do_constant_folding=True ) # whether to execute constant folding for optimization
    #                   dynamic_axes={'input' : {0 : 'batch_size'},    # variable length axes
    #                                 'output' : {0 : 'batch_size'}})

    # Run torch model
    torch_out=model(dummy_input)

# onnx_model = onnx.load("model_g.onnx")
# onnx.checker.check_model(onnx_model, True)
ort_session = onnxruntime.InferenceSession("model_gdnet.onnx") #providers=["CPUExecutionProvider"]

def to_numpy(tensor):
    return tensor.detach().cpu().numpy() if tensor.requires_grad else tensor.cpu().numpy()

# compute ONNX Runtime output prediction
ort_inputs = {ort_session.get_inputs()[0].name: to_numpy(dummy_input)}
ort_outs = ort_session.run(None, ort_inputs)

# compare ONNX Runtime and PyTorch results
np.testing.assert_allclose(to_numpy(torch_out[0]), ort_outs[0], rtol=1e-03, atol=1e-05)

print("Exported model has been tested with ONNXRuntime, and the result looks good!")

/usr/local/lib/python3.10/dist-packages/torch/nn/functional.py:3782: UserWarning: nn.functional.upsample is deprecated. Use nn.functional.interpolate instead.
  warnings.warn("nn.functional.upsample is deprecated. Use nn.functional.interpolate instead.")


Exported model has been tested with ONNXRuntime, and the result looks good!


In [ ]:
from google.colab import files
files.download('model_gdnet.onnx')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>